In [4]:
import collections
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import re
import requests
import time
import xml.etree.ElementTree

In [5]:
st_accept = "text/html"
st_useragent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"
headers = {
   "Accept": st_accept,
   "User-Agent": st_useragent
}

In [ ]:
ITGAS_BASE_LINK = "https://it-gas.ru/"
ITGAS_LINKS = ["oxygen", "propane", "acetylene", "argon", "nitrogen", "carbon-dioxide", "helium",
               "welding-gas-mixtures", "food-gas-mixtures", "app-table", "mixtures-name", "technic-welding",
               "specifications-gas-mixtures", "mig-mag-tig", ""]
PANDAS_CRUTCH = "/content/temp.txt"

#Метод find_all() рекурсивно обходит дерево тегов и возвращает в виде списка
#все теги дерева. По этой причине дублируется содержимое вложенных тегов,
#например, списков. Чтобы это не происходило, сохраняем во множестве id
#обработанных тегов и каждый раз проверяем, обработан ли тег.
processed_tags = {}

def parse_ul(ul):
    ul_elements = ul.children

    for i in ul_elements:
        if i.name == None or id(i) in processed_tags:
            continue

        processed_tags.add(id(i))

        #Парсер видит символы в нижнем индексе в химических формулах
        #как отдельных потомков, потому что используется тег <sub></sub>.
        #Этих потомков игнорируем, чтобы не дублировать текст.
        subelements = i.children
        sublist = None
        for j in subelements:
            if j.name == "sub":
                j.decompose()
            elif j.name == "ul":
                sublist = j

        if sublist == None:
            f.write("        " + re.sub(" +", " ", i.get_text().lstrip().rstrip()) + "\n")
        else:
            parse_ul(sublist)
    f.write("\n")
    processed_tags.add(ul)
    return

for i in ITGAS_LINKS:
    req = requests.get(ITGAS_BASE_LINK + i, headers)
    soup = bs(req.text, 'lxml')
    processed_tags = set()

    #Костыль для Pandas. Скидываем полученный html в файл и ищем в нём все таблицы.
    temp_file = open("temp.txt", 'w', encoding="utf-8")
    temp_file.write(req.text)
    temp_file.close()
    tables = pd.read_html(PANDAS_CRUTCH)
    table_index = 0

    page_title = soup.find("div", attrs={"class":"page-title"})

    element_letter = soup.find("div", attrs={"class":"chW-chemical"})
    element_name = soup.find("div", attrs={"class":"chW-desc"})

    f = open(page_title.text + ".txt", 'w', encoding="utf-8")
    if element_name != None:
        f.write(element_name.text + " - " + element_letter.text + "\n")

    page_info = soup.find("div", attrs={"class":"float-t"}).find_all()

    for j in page_info:
        if id(j) in processed_tags:
            continue

        if j.name == "h2":
            f.write(j.text + "\n")
        elif j.name == "h3":
            f.write("    " + j.text + ":\n")
        elif j.name == "p":
            f.write(re.sub(" +", " ", j.text.lstrip()) + "\n")
        elif j.name == "ul":
            parse_ul(j)
        elif j.name == "table":
            if "clr-tbl" not in j["class"]:
                f.write(tables[table_index].fillna(value="").map(lambda l: l.replace("?", "/")).to_string(index=False,header=False) + "\n\n")
            table_index += 1
        else:
            pass

        processed_tags.add(id(j))

    #Закрываем файл, чистим список обработанных тегов.
    f.close()
    processed_tags.clear()

    #Можно накинуть задержку, чтобы не ддосить сайт.
    time.sleep(4)

#Убираем мусор.
os.remove(PANDAS_CRUTCH)